In [2]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.context import SparkContext
from pyspark.sql import functions as F


In [3]:
spark = SparkSession.builder.appName("Interview").getOrCreate()

24/04/24 02:05:33 WARN Utils: Your hostname, Shivanis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.106 instead (on interface en0)
24/04/24 02:05:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/24 02:05:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 49284)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 761, in __init__
    self.handle()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Library/Frameworks/Pyt

In [ ]:
''' Question 1. Write a PySpark Query using below input to get below output.  #KPMG
Input 
+-----+---------------------+
|  Name|             Hobbies|
+-----+---------------------+
| Alice|  Badmintion, Tennis|
|   Bob|     Tennis, Cricket|
| Julie|    Cricket, Carroms|
+-----+---------------------+
Output
+-----+------------+
|  Name|    Hobbies|
+-----+------------+
| Alice| Badmintion|
| Alice|     Tennis|
|   Bob|     Tennis|
|   Bob|    Cricket|
| Julie|    Cricket|
| Julie|    Carroms|
+-----+------------+

'''


data = [('Alice', 'Badminton, Tennis' ), ('Bob',  'Tennis, Circket'), ('Julie', 'Cricket, Carroms')]
columns = ["Name", "Hobbies"]
df1 = spark.createDataFrame(data, columns)
df1.show()

#split : In PySpark, the split() function is used to split a string column into an array of substrings based on a delimiter.
#explode : function is used to split a column that contains an array or map type into multiple rows, with each element of the array or map occupying its own row. 
from pyspark.sql.functions import split, explode
df1= df1.select(df1.Name, explode(split(df1.Hobbies, ',')).alias("Hobbies"))
df1.show()

In [ ]:
'''Question2. Write a PySpark Query using below input to get below output.  #KPMG
Input
+-------+-------+-------+
| City 1| City 2| City 3|
+-------+-------+-------+
|    Goa|       |     AP|
|       |     AP|   null|
|   null|       |   Bglu|
+-------+-------+-------+

Output
+-------+
| Result| 
+-------+
|    Goa|
|     AP| 
|   Bglu|
+-------+

'''

df2 = data = [('Goa', '', 'AP' ), ('',  'AP', None), (None, '', 'Bglu')]
columns = ["City1", "City2", "City3"]
df2 = spark.createDataFrame(data, columns)
df2.show()


#coalesce : In PySpark, the coalesce() function is used to return the first non-null value among the input columns. 
#It takes multiple columns as input and returns a new column with the first non-null value from those columns.
from pyspark.sql.functions import coalesce,when
df2 = df2.withColumn('Result', coalesce(
    when(df2.City1 == "", None).otherwise(df2.City1), 
    when(df2.City2 == "", None).otherwise(df2.City2), 
    when(df2.City3 == "", None).otherwise(df2.City3)
    )).drop(df2.City1, df2.City2, df2.City3)
df2.show()


In [20]:
'''Question3. You have 2 Dataframe as Input and you have to get the below Output 
Input 
+---+-----+         +---+-------+----+
| Id| Name|         | Id|Subject|Mark|
+---+-----+         +---+-------+----+
|  1|Steve|         |  1|    SQL|  90|
|  2|David|         |  1|PySpark| 100|
|  3| John|         |  2|    SQL|  70|
|  4|Shree|         |  2|PySpark|  60|
|  5|Helen|         |  3|    SQL|  30|
+---+-----+         |  3|PySpark|  20|
                    |  4|    SQL|  50|
                    |  4|PySpark|  50|
                    |  5|    SQL|  45|
                    |  5|PySpark|  45|
                    +---+-------+----+

Output
+---+-----+----------+------------+
| Id| Name|Percentage|      Result|
+---+-----+----------+------------+
|  1|Steve|      95.0|  Distintion|
|  2|David|      65.0| First Class|
|  3| John|      25.0|        Fail|
|  4|Shree|      50.0|Second Class|
|  5|Helen|      45.0| Third Class|
+---+-----+----------+------------+                    

'''

data1=[(1,"Steve"),(2,"David"),(3,"John"),(4,"Shree"),(5,"Helen")]
data2=[(1,"SQL",90),(1,"PySpark",100),(2,"SQL",70),(2,"PySpark",60),(3,"SQL",30),(3,"PySpark",20),(4,"SQL",50),(4,"PySpark",50),(5,"SQL",45),(5,"PySpark",45)]
schema1=["Id","Name"]
schema2=["Id","Subject","Mark"]

student_df = spark.createDataFrame(data1,schema1)
student_df.show()
marks_df = spark.createDataFrame(data2,schema2)
joined_df = student_df.join(marks_df, student_df.Id == marks_df.Id).drop(marks_df.Id)

from pyspark.sql.functions import *
joined_df = joined_df.groupBy('Id', 'Name').agg((sum('Mark')/count('*')).alias("Percentage"))

result_df = joined_df.select('*', 
                             (when(joined_df.Percentage >= 70, 'Distintion')
                             .when((joined_df.Percentage < 70) &  (joined_df.Percentage >=60), 'First Class')
                             .when((joined_df.Percentage < 60) &  (joined_df.Percentage >=50), 'Second Class')
                             .when((joined_df.Percentage < 50) & (joined_df.Percentage >= 40), 'Third Class')
                             .when((joined_df.Percentage <=39), 'Fail')).alias("Result")
                             )

result_df.show()

+---+-----+
| Id| Name|
+---+-----+
|  1|Steve|
|  2|David|
|  3| John|
|  4|Shree|
|  5|Helen|
+---+-----+

+---+-----+----------+------------+
| Id| Name|Percentage|      Result|
+---+-----+----------+------------+
|  1|Steve|      95.0|  Distintion|
|  2|David|      65.0| First Class|
|  3| John|      25.0|        Fail|
|  4|Shree|      50.0|Second Class|
|  5|Helen|      45.0| Third Class|
+---+-----+----------+------------+



In [27]:
'''Question4. Employee Who are getting Higest salary in their department
Input
+-----+-------+------+--------+
|EmpId|EmpName|Salary|DeptName|
+-----+-------+------+--------+
|    1|   Aman|  1000|      IT|
|    2|    Bob|  1500|      IT|
|    3|    Col|  2500|      IT|
|    4|    Den|  3000|      HR|
|    5|  Eimly|  2000|      HR|
|    6|    Fan|  1000|      HR|
|    7|   Goer|  4000|   Sales|
|    8|    Hen|  4000|   Sales|
|    9|   Iton|  1000|   Sales|
|   10|   Jack|  2000|   Sales|
+-----+-------+------+--------+

Output 
-----+-------+------+--------+
|EmpId|EmpName|Salary|DeptName|
+-----+-------+------+--------+
|    3|    Col|  2500|      IT|
|    4|    Den|  3000|      HR|
|    7|   Goer|  4000|   Sales|
|    8|    Hen|  4000|   Sales|
+-----+-------+------+--------+

'''
data=[(1,"Aman",1000,"IT"),(2,"Bob",1500,"IT"),(3,"Col",2500,"IT"),(4,"Den",3000,"HR"),(5,"Eimly",2000,"HR"),(6,"Fan",1000,"HR")
       ,(7,"Goer",4000,"Sales"),(8,"Hen",4000,"Sales"),(9,"Iton",1000,"Sales"),(10,"Jack",2000,"Sales")]
schema=["EmpId","EmpName","Salary","DeptName"]
df=spark.createDataFrame(data,schema)

# df = df.groupby('DeptName').agg(max('Salary').alias('Salary'))

from pyspark.sql.functions import *
from pyspark.sql.window import *


df= df.select('*', dense_rank().over(Window.partitionBy(df.DeptName).orderBy(df.Salary.desc())).alias("Rank"))
df = df.filter(df.Rank==1).drop(df.Rank)

df.show()

+-----+-------+------+--------+
|EmpId|EmpName|Salary|DeptName|
+-----+-------+------+--------+
|    4|    Den|  3000|      HR|
|    3|    Col|  2500|      IT|
|    7|   Goer|  4000|   Sales|
|    8|    Hen|  4000|   Sales|
+-----+-------+------+--------+



In [45]:
'''
Question 5.  Given 2 Input DataFram find out the 
1. Create a Dataframe with these columns with the given Input :-> DeptName, MgrName, EmpName, SalaryYear, SalaryMonth, TotalMonthlySalary
2. 7th Higest Salaries and Show EmpName
#KPMG

Input DataFrames
df_salary                                                   df_dept
+-----+-------+-----+------+----------+------+              +------+--------+           
|EmpId|EmpName|MgrId|DeptId|SalaryDate|Salary|              |DeptId|DeptName|
+-----+-------+-----+------+----------+------+              +------+--------+
|  100|    Raj| NULL|     1|  01-04-23| 50000|              |     1|      IT|
|  200| Joanne|  100|     1|  01-04-23|  4000|              |     2|      HR|
|  200| Joanne|  100|     1|  13-04-23|  4500|              +------+--------+
|  200| Joanne|  100|     1|  14-04-23|  4020|
+-----+-------+-----+------+----------+------+    
'''

data1=[(100,"Raj",None,1,"01-04-23",50000), (200,"Joanne",100,1,"01-04-23",4000),(200,"Joanne",100,1,"13-04-23",4500),(200,"Joanne",100,1,"14-04-23",4020)]
schema1=["EmpId","EmpName","MgrId","DeptId","SalaryDate","Salary"]
df_salary=spark.createDataFrame(data1,schema1)
data2=[(1,"IT"),(2,"HR")]
schema2=["DeptId","DeptName"]
df_dept=spark.createDataFrame(data2,schema2)

df = df_salary.withColumn("NewSalaryDate", to_date('SalaryDate', 'dd-mm-yy')).drop(df_salary.SalaryDate)
df = df.join(df_dept, df_dept.DeptId == df.DeptId).drop(df_dept.DeptId )

df = df.alias('a').join(df.alias('b'), col('a.MgrId') == col('b.EmpId'), 'left').select(
    col('a.DeptName'), 
    col('b.EmpName').alias('MgrName'),
    col('a.EmpName'),
    col('a.NewSalaryDate'),
    col('a.Salary')
)

df = df.groupBy('DeptName', 'MgrName', 'EmpName', year('NewSalaryDate').alias('SalaryYear'), date_format('NewSalaryDate', 'MMMM').alias('SalaryMonth')).sum('Salary')
df.show()

+--------+-------+-------+----------+-----------+-----------+
|DeptName|MgrName|EmpName|SalaryYear|SalaryMonth|sum(Salary)|
+--------+-------+-------+----------+-----------+-----------+
|      IT|    Raj| Joanne|      2023|    January|      12520|
|      IT|   NULL|    Raj|      2023|    January|      50000|
+--------+-------+-------+----------+-----------+-----------+



24/04/24 03:59:38 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 473332 ms exceeds timeout 120000 ms
24/04/24 03:59:38 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/24 03:59:42 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$